In [30]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


##### Define Foursquare Credentials and Version

In [38]:
# @hidden_cell
CLIENT_ID = 'V53MHPBBL23EVUFTF31HTKJAFRJIK2QL1WJURTZ3ARRFF3KE' # Foursquare ID
CLIENT_SECRET = 'G5Z0ALDAAKEOPESPXKBZPXNB2TIEJKGWM33ZGSBCNW0VGW0T' # Foursquare Secret
VERSION = '20180604'
LIMIT = 100

#### Explore venues around my work place 

In [25]:
address = 'Russia, Taganrog'
#address = 'Россия, Таганрог, Греческая 104'
#address = 'Russia, Taganrog, Grecheskaya 104'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

47.2153657 38.9285216


In [49]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            500, 
            LIMIT)

In [33]:
import json, codecs
def json_to_file(data, name='work_nbrs.json'):
    with open(name, 'wb') as f:
        json.dump(data, codecs.getwriter('utf-8')(f), ensure_ascii=False)

In [70]:
def getVenuesWithin(sw, ne):
    
    venues_list=[]
              
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&intent=browse&sw={},{}&ne={},{}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            sw[0], 
            sw[1], 
            ne[0], 
            ne[1],
            LIMIT)
            
    # make the GET request
    results = requests.get(url).json()['response']['venues']
        
    # return only relevant information for each nearby venue
    venues_list.append([(
        v['name'], 
        v['location']['lat'], 
        v['location']['lng'],  
        v['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
        'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)
        

In [71]:
sw = [47.212427, 38.925331]
ne = [47.286151, 38.959084]
venues = getVenuesWithin(sw=sw, ne=ne)

IndexError: list index out of range

In [ ]:
venues.head()

#### Searching all venues in the Tagarog city

In [82]:
address = 'Russia, Taganrog'
loc = geolocator.geocode(address)
sw = [47.212427, 38.925331]
ne = [47.286151, 38.959084]
lat = ne[0]
lon = ne[1]
search_url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&intent=browse&sw={},{}&ne={},{}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            #address,
            sw[0],
            sw[1],
            ne[0],
            ne[1],
            500)
search_url

'https://api.foursquare.com/v2/venues/search?&client_id=V53MHPBBL23EVUFTF31HTKJAFRJIK2QL1WJURTZ3ARRFF3KE&client_secret=G5Z0ALDAAKEOPESPXKBZPXNB2TIEJKGWM33ZGSBCNW0VGW0T&v=20180604&intent=browse&sw=47.212427,38.925331&ne=47.286151,38.959084&limit=500'

In [90]:
#results = requests.get(search_url).json()['response']['venues']
i = 0
for v in results:
    print(i)
    print(v['categories'][0]['name'])
    i = i + 1



0
Beach
1
Historic Site
2
Harbor / Marina
3
Beach
4
Water Park
5
Gas Station
6
Gift Shop
7
Hotel
8
Doctor's Office
9
Pharmacy
10
Train
11
Cosmetics Shop
12
Modern European Restaurant
13
Shoe Store
14
Clothing Store
15
Mobile Phone Shop
16
Accessories Store
17
Hardware Store
18
Middle School
19
Clothing Store
20
Pharmacy
21
Clothing Store
22
Clothing Store
23
Miscellaneous Shop
24
Pawn Shop
25
Salon / Barbershop
26
Mobile Phone Shop
27
Electronics Store
28
Fast Food Restaurant
29
Medical Center
30
Eastern European Restaurant
31
General Travel
32
Café
33
Residential Building (Apartment / Condo)
34
Monument / Landmark
35
Hospital
36
Non-Profit
37
Office
38
Office
39
Bridal Shop
40
Butcher
41
Salon / Barbershop
42
Movie Theater
43
Strip Club
44
Gym / Fitness Center
45
Playground
46
Restaurant
47
Office
48
Military Base
49
Residential Building (Apartment / Condo)
50
Office
51
Health Food Store
52
Building
53
Pharmacy
54
Sandwich Place
55
Fast Food Restaurant
56
Road
57
Nightclub
58
Pool
59


IndexError: list index out of range

In [62]:

json_to_file(results)

In [92]:
results[72]

{'id': '52660ddd11d2c3020869f610',
 'name': 'металлургический завод',
 'location': {'lat': 47.25108329496104,
  'lng': 38.93420037179856,
  'labeledLatLngs': [{'label': 'display',
    'lat': 47.25108329496104,
    'lng': 38.93420037179856}],
  'cc': 'RU',
  'city': 'Taganrog',
  'state': "Yuzhniy Federal'niy Okrug",
  'country': 'Россия',
  'formattedAddress': ['Taganrog', 'Россия']},
 'categories': [],
 'referralId': 'v-1571843858',
 'hasPerk': False}

In [84]:
venues_list = []
venues_list.append([(
        v['name'], 
        v['location']['lat'], 
        v['location']['lng'],  
        v['categories'][0]['name']) for v in results])

IndexError: list index out of range

In [85]:
venues_list

[]